### Persistence, Time Travel etc

In [45]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [ ]:
load_dotenv()
api_key = ""
llm = ChatOpenAI(api_key=api_key)

In [47]:
##  State
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [48]:
## generate_joke function
def generate_joke(state: JokeState):

    prompt = f"generate a joke on the topic {state['topic']}"
    response = llm.invoke(prompt).content

    return {'joke':  response}

In [49]:
## generate_explanation function
def generate_explanation(state: JokeState):

    prompt = f"write an explanation for the joke - {state['joke']}"
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [50]:
## graph building
graph = StateGraph(JokeState)

## adding nodes
graph.add_node('generate_joke',  generate_joke)
graph.add_node('generate_explanation', generate_explanation)


## adding edges 
graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [51]:
config1 = {"configurable": {"thread_id": '1'}}
workflow.invoke({'topic': 'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza go to the party? Because it wanted to get a slice of the action!',
 'explanation': 'This joke is a play on words, using the phrase "slice of the action" in a literal sense. When someone says they want to get a "slice of the action," it means they want to be involved in or participate in something exciting or worthwhile. In this joke, the pizza goes to the party because it literally wants to get a slice (piece) of the action, as in a slice of pizza. It\'s a lighthearted and punny way to explain why the pizza decided to attend the party.'}

In [52]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party? Because it wanted to get a slice of the action!', 'explanation': 'This joke is a play on words, using the phrase "slice of the action" in a literal sense. When someone says they want to get a "slice of the action," it means they want to be involved in or participate in something exciting or worthwhile. In this joke, the pizza goes to the party because it literally wants to get a slice (piece) of the action, as in a slice of pizza. It\'s a lighthearted and punny way to explain why the pizza decided to attend the party.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07ccca-f14c-6f79-8002-cad4a51b89b4'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-19T07:18:20.103461+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07ccca-ab46-684a-8001-6e02ce852de7'}}, tasks=(), interrupts=())

In [53]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party? Because it wanted to get a slice of the action!', 'explanation': 'This joke is a play on words, using the phrase "slice of the action" in a literal sense. When someone says they want to get a "slice of the action," it means they want to be involved in or participate in something exciting or worthwhile. In this joke, the pizza goes to the party because it literally wants to get a slice (piece) of the action, as in a slice of pizza. It\'s a lighthearted and punny way to explain why the pizza decided to attend the party.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07ccca-f14c-6f79-8002-cad4a51b89b4'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-19T07:18:20.103461+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07ccca-ab46-684a-8001-6e02ce852de7'}}, tasks=(), interrupts=())

In [55]:
config2 = {'configurable': {'thread_id': '2'}}
workflow.invoke({'topic': "pasta"}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti go to the party?\n\nBecause it heard it would be a pasta-tively great time!',
 'explanation': 'This joke is a play on words using a pun. The word "pasta-tively" is a combination of "pasta" and "positively," suggesting that the spaghetti thought the party would be a great time because it heard it would be filled with pasta dishes. It\'s a lighthearted and funny way to imagine a plate of spaghetti attending a party.'}

In [56]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party? Because it wanted to get a slice of the action!', 'explanation': 'This joke is a play on words, using the phrase "slice of the action" in a literal sense. When someone says they want to get a "slice of the action," it means they want to be involved in or participate in something exciting or worthwhile. In this joke, the pizza goes to the party because it literally wants to get a slice (piece) of the action, as in a slice of pizza. It\'s a lighthearted and punny way to explain why the pizza decided to attend the party.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07ccca-f14c-6f79-8002-cad4a51b89b4'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-19T07:18:20.103461+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07ccca-ab46-684a-8001-6e02ce852de7'}}, tasks=(), interrupts=())

In [58]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party? Because it wanted to get a slice of the action!', 'explanation': 'This joke is a play on words, using the phrase "slice of the action" in a literal sense. When someone says they want to get a "slice of the action," it means they want to be involved in or participate in something exciting or worthwhile. In this joke, the pizza goes to the party because it literally wants to get a slice (piece) of the action, as in a slice of pizza. It\'s a lighthearted and punny way to explain why the pizza decided to attend the party.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07ccca-f14c-6f79-8002-cad4a51b89b4'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-19T07:18:20.103461+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07ccca-ab46-684a-8001-6e02ce852de7'}}, tasks=(), interrupts=())

### Time Travel

In [59]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f07ccca-82e0-6a2d-8000-28cbc4f7665c"}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f07ccca-82e0-6a2d-8000-28cbc4f7665c'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-08-19T07:18:08.524753+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07ccca-82de-629e-bfff-1955d284bdcc'}}, tasks=(PregelTask(id='05301c83-be76-25f8-0e91-9263ca146319', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pizza go to the party? Because it wanted to get a slice of the action!'}),), interrupts=())

In [60]:
## executing from the mentioned checkpoint
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f07ccca-82e0-6a2d-8000-28cbc4f7665c"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza go to the doctor? Because it was feeling saucy!',
 'explanation': 'This joke plays on the double meaning of the word "saucy." On one hand, "saucy" can mean bold, impudent, or cheeky, which is a playful way to describe the attitude of someone or something. On the other hand, "saucy" can also refer to having a lot of sauce, as in the case of a pizza being covered in tomato sauce. So, the joke suggests that the pizza went to the doctor because it was feeling bold or cheeky (saucy), while also hinting at the fact that it may have had too much sauce on it.'}

In [61]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it was feeling saucy!', 'explanation': 'This joke plays on the double meaning of the word "saucy." On one hand, "saucy" can mean bold, impudent, or cheeky, which is a playful way to describe the attitude of someone or something. On the other hand, "saucy" can also refer to having a lot of sauce, as in the case of a pizza being covered in tomato sauce. So, the joke suggests that the pizza went to the doctor because it was feeling bold or cheeky (saucy), while also hinting at the fact that it may have had too much sauce on it.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07cd4a-f0d7-6bab-8002-3cc4b2119eff'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-19T08:15:36.029277+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07cd4a-db08-617f-8001-946cdc9d8051'}}, tasks=()

### updating state

In [63]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f07ccca-82e0-6a2d-8000-28cbc4f7665c", "checkpoint_ns": ""}}, {'topic': 'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f07cd57-0e64-6169-8001-ce19c7f67a6c'}}

In [64]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07cd57-0e64-6169-8001-ce19c7f67a6c'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-08-19T08:21:01.250186+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07ccca-82e0-6a2d-8000-28cbc4f7665c'}}, tasks=(PregelTask(id='9c1bee41-31f2-8bd4-9fad-a055c1b2a37f', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it was feeling saucy!', 'explanation': 'This joke plays on the double meaning of the word "saucy." On one hand, "saucy" can mean bold, impudent, or cheeky, which is a playful way to describe the attitude of someone or something. On the other hand, "saucy" can also refer to having a lot

In [65]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f07cd57-0e64-6169-8001-ce19c7f67a6c"}})

{'topic': 'samosa',
 'joke': "Why couldn't the samosa make it to the party on time? Because it was stuck in a pastry traffic jam!",
 'explanation': 'This joke is a play on words, using the term "pastry traffic jam" as a pun on a real traffic jam. In this case, the samosa was unable to make it to the party on time because it was figuratively stuck in a pastry traffic jam, likely referencing other pastries or foods that were in its way and preventing it from getting to its destination quickly. The humorous aspect of the joke lies in the unexpected twist of a samosa being held up by other pastries, creating a silly and lighthearted image in the reader\'s mind.'}

In [66]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': "Why couldn't the samosa make it to the party on time? Because it was stuck in a pastry traffic jam!", 'explanation': 'This joke is a play on words, using the term "pastry traffic jam" as a pun on a real traffic jam. In this case, the samosa was unable to make it to the party on time because it was figuratively stuck in a pastry traffic jam, likely referencing other pastries or foods that were in its way and preventing it from getting to its destination quickly. The humorous aspect of the joke lies in the unexpected twist of a samosa being held up by other pastries, creating a silly and lighthearted image in the reader\'s mind.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07cd5d-d274-6231-8003-9bc3d50a619f'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-08-19T08:24:02.870123+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'check

In [67]:
### end here